# Bootstraping

In [36]:
import Pkg

Pkg.add("CovarianceMatrices")
Pkg.add("Distributions")
Pkg.add("DataFrames")
Pkg.add("CSV")
Pkg.add("Tables")
Pkg.add("GLM")
Pkg.add("Gadfly")
Pkg.add("RData")
Pkg.add("CodecBzip2")
Pkg.add("DataStructures")
Pkg.add("NamedArrays")
Pkg.add("PrettyTables")
Pkg.add("Lasso")
Pkg.add("CategoricalArrays")

using Lasso
using Distributions
using DataFrames
using CSV
using Tables
using GLM
using CovarianceMatrices
using RData, LinearAlgebra, GLM, DataFrames, Statistics, Random, Distributions, DataStructures, NamedArrays, PrettyTables
using CategoricalArrays
import CodecBzip2

using Gadfly

   Resolving package versions...
  No Changes to `C:\Users\Franc\.julia\environments\v1.10\Project.toml`
  No Changes to `C:\Users\Franc\.julia\environments\v1.10\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\Franc\.julia\environments\v1.10\Project.toml`
  No Changes to `C:\Users\Franc\.julia\environments\v1.10\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\Franc\.julia\environments\v1.10\Project.toml`
  No Changes to `C:\Users\Franc\.julia\environments\v1.10\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\Franc\.julia\environments\v1.10\Project.toml`
  No Changes to `C:\Users\Franc\.julia\environments\v1.10\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\Franc\.julia\environments\v1.10\Project.toml`
  No Changes to `C:\Users\Franc\.julia\environments\v1.10\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\Franc\.julia\environments\v1.10\Project.toml`
  No 

In [54]:
# Load the dataset
data = CSV.read("penn_jae.dat", DataFrame; delim=' ', header=true,  silencewarnings=true)

# Define the column names
column_names = [:abdt, :tg, :inuidur1, :inuidur2, :female, :black, :hispanic, :othrace, :dep, :q1, :q2, :q3, :q4, :q5, :q6, :recall, :agelt35, :agegt54, :durable, :nondurable, :lusd, :husd, :muld]

# Rename the columns manually to ensure they match the expected names
rename!(data, names(data) .=> column_names)

# Display the first few rows of the dataset to confirm it is loaded correctly
first(data, 5)

Row,abdt,tg,inuidur1,inuidur2,female,black,hispanic,othrace,dep,q1,q2,q3,q4,q5,q6,recall,agelt35,agegt54,durable,nondurable,lusd,husd,muld
,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64
1,10824,0,18,18,0,0,0,0,2,0,0,0,0,1,0,0,0,0,0,0,0,1,0
2,10635,2,7,3,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0
3,10551,5,18,6,1,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0
4,10824,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0
5,10747,0,27,27,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0


In [55]:
# Display the column names to confirm they are set correctly
println(names(data))

["abdt", "tg", "inuidur1", "inuidur2", "female", "black", "hispanic", "othrace", "dep", "q1", "q2", "q3", "q4", "q5", "q6", "recall", "agelt35", "agegt54", "durable", "nondurable", "lusd", "husd", "muld"]


In [56]:
# Create columns to separate "dep" into dummies
data.dep_0 = Int.(data.dep .== 0)
data.dep_1 = Int.(data.dep .== 1)

data

Row,abdt,tg,inuidur1,inuidur2,female,black,hispanic,othrace,dep,q1,q2,q3,q4,q5,q6,recall,agelt35,agegt54,durable,nondurable,lusd,husd,muld,dep_0,dep_1
,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64
1,10824,0,18,18,0,0,0,0,2,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0
2,10635,2,7,3,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,1,0
3,10551,5,18,6,1,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,1,0
4,10824,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0
5,10747,0,27,27,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0
6,10544,6,7,7,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0
7,10845,1,1,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,1,0
8,10670,3,3,3,1,0,0,0,2,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0
9,10768,3,28,11,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1,0


In [60]:
# Define the regression formula
formula = @formula log(inuidur1) ~ tg + female + black + othrace + q2 + q3 + q4 + q5 + q6 + agelt35 + agegt54 + durable + lusd + husd + dep_0 + dep_1

# Function to perform a single bootstrap sample and regression
function bootstrap_sample(data, formula)
    sample_data = data[rand(1:nrow(data), nrow(data)), :]
    model = lm(formula, sample_data)
    return coef(model)
end

# Number of bootstrap samples
n_bootstrap = 1000

1000

In [61]:
# Store the bootstrap estimates
bootstrap_estimates = DataFrame(T4=Float64[], female=Float64[], black=Float64[])

# Perform the bootstrap procedure
for i in 1:n_bootstrap
    coefs = bootstrap_sample(data, formula)
    push!(bootstrap_estimates, (T4=coefs[2], female=coefs[3], black=coefs[4]))
end

In [62]:
# Calculate standard errors of the bootstrap estimates
bootstrap_se = describe(bootstrap_estimates, :std)[!, "std"]

# Present results in a table
results = DataFrame(
    Variable = ["T4", "female", "black"],
    StandardError = bootstrap_se
)

println(results)

3×2 DataFrame
 Row │ Variable  StandardError 
     │ String    Float64       
─────┼─────────────────────────
   1 │ T4           0.00468849
   2 │ female       0.0210793
   3 │ black        0.0376854
